In [14]:
import sys
sys.path.append("..")

In [55]:
from collections import defaultdict

import numpy as np
import pandas as pd
import torch

# Exploring vehicle counter data

In [37]:
ls ../data/train/london/input | tail -n 5

counters_2020-01-27.parquet
counters_2020-01-28.parquet
counters_2020-01-29.parquet
counters_2020-01-30.parquet
counters_2020-01-31.parquet


In [63]:
df = pd.read_parquet("../data/train/london/input/counters_2020-01-27.parquet")

In [64]:
df.head()

,node_id,day,t,volumes_1h
2396,101818,2020-01-27,0,"[nan, 84.0, 96.0, 58.0]"
2397,101818,2020-01-27,1,"[84.0, 96.0, 58.0, 54.0]"
2398,101818,2020-01-27,2,"[96.0, 58.0, 54.0, nan]"
2399,101818,2020-01-27,3,"[58.0, 54.0, nan, nan]"
2400,101818,2020-01-27,4,"[54.0, nan, nan, nan]"


In [65]:
df.shape

(335328, 4)

In [66]:
df[(df.day == "2020-01-27") & (df.t == 1)]

,node_id,day,t,volumes_1h
2397,101818,2020-01-27,1,"[84.0, 96.0, 58.0, 54.0]"
5369,101831,2020-01-27,1,"[182.0, 140.0, 109.0, 104.0]"
8341,101839,2020-01-27,1,"[117.0, 96.0, 80.0, 84.0]"
11313,101843,2020-01-27,1,"[212.0, 194.0, 157.0, 151.0]"
14285,101870,2020-01-27,1,"[502.0, 465.0, 386.0, 298.0]"
...,...,...,...,...
10309505,9160193909980290120,2020-01-27,1,"[247.0, 249.0, 252.0, 222.0]"
10312477,9179255819075450292,2020-01-27,1,"[0.0, 0.0, 0.0, 0.0]"
10315449,9179908236179360682,2020-01-27,1,"[238.0, 253.0, 246.0, 205.0]"
10318421,9193791959321534875,2020-01-27,1,"[308.0, 281.0, 267.0, 259.0]"


In [9]:
df.head().explode("volumes_1h")

,node_id,day,t,volumes_1h
0,101818,2019-07-01,4,NaN
0,101818,2019-07-01,4,NaN
0,101818,2019-07-01,4,NaN
0,101818,2019-07-01,4,NaN
1,101818,2019-07-01,5,NaN
1,101818,2019-07-01,5,NaN
1,101818,2019-07-01,5,NaN
1,101818,2019-07-01,5,NaN
2,101818,2019-07-01,6,NaN
2,101818,2019-07-01,6,NaN


In [12]:
ls data/speed_classes/london | head -n 5

speed_classes_2019-07-01.parquet
speed_classes_2019-07-02.parquet
speed_classes_2019-07-03.parquet
speed_classes_2019-07-04.parquet
speed_classes_2019-07-05.parquet


In [13]:
pd.read_parquet("data/speed_classes/london/speed_classes_2019-07-01.parquet").head()

,u,v,day,t,volume_class,median_speed_kph,free_flow_kph
0,78112,25508583,2019-07-01,9,3,19.764706,36.352941
1,78112,25508583,2019-07-01,28,5,27.882353,36.352941
2,78112,25508583,2019-07-01,29,5,46.823529,36.352941
3,78112,25508583,2019-07-01,30,3,24.000000,36.352941
4,78112,25508583,2019-07-01,34,5,38.470588,36.352941


In [ ]:
ls data/train/london/labels

cc_labels_2019-07-01.parquet  cc_labels_2019-10-13.parquet
cc_labels_2019-07-02.parquet  cc_labels_2019-10-21.parquet
cc_labels_2019-07-03.parquet  cc_labels_2019-10-22.parquet
cc_labels_2019-07-04.parquet  cc_labels_2019-10-23.parquet
cc_labels_2019-07-05.parquet  cc_labels_2019-10-24.parquet
cc_labels_2019-07-06.parquet  cc_labels_2019-10-25.parquet
cc_labels_2019-07-07.parquet  cc_labels_2019-10-26.parquet
cc_labels_2019-07-15.parquet  cc_labels_2019-10-27.parquet
cc_labels_2019-07-16.parquet  cc_labels_2019-11-04.parquet
cc_labels_2019-07-17.parquet  cc_labels_2019-11-05.parquet
cc_labels_2019-07-18.parquet  cc_labels_2019-11-06.parquet
cc_labels_2019-07-19.parquet  cc_labels_2019-11-07.parquet
cc_labels_2019-07-20.parquet  cc_labels_2019-11-08.parquet
cc_labels_2019-07-21.parquet  cc_labels_2019-11-09.parquet
cc_labels_2019-07-29.parquet  cc_labels_2019-11-10.parquet
cc_labels_2019-07-30.parquet  cc_labels_2019-11-18.parquet
cc_labels_2019-07-31.parquet  cc_labels_2019-11-19.parqu

In [8]:
df = pd.read_parquet("data/train/london/labels/cc_labels_2019-07-01.parquet")

In [9]:
df.head()

,u,v,day,t,cc
0,78112,25508583,2019-07-01,9,2
1,78112,25508583,2019-07-01,28,1
2,78112,25508583,2019-07-01,29,1
3,78112,25508583,2019-07-01,30,2
4,78112,25508583,2019-07-01,34,1


In [11]:
ls data/train/madrid/labels

cc_labels_2021-06-01.parquet  cc_labels_2021-09-13.parquet
cc_labels_2021-06-02.parquet  cc_labels_2021-09-21.parquet
cc_labels_2021-06-03.parquet  cc_labels_2021-09-22.parquet
cc_labels_2021-06-04.parquet  cc_labels_2021-09-23.parquet
cc_labels_2021-06-05.parquet  cc_labels_2021-09-24.parquet
cc_labels_2021-06-06.parquet  cc_labels_2021-09-25.parquet
cc_labels_2021-06-07.parquet  cc_labels_2021-09-26.parquet
cc_labels_2021-06-15.parquet  cc_labels_2021-09-27.parquet
cc_labels_2021-06-16.parquet  cc_labels_2021-10-05.parquet
cc_labels_2021-06-17.parquet  cc_labels_2021-10-06.parquet
cc_labels_2021-06-18.parquet  cc_labels_2021-10-07.parquet
cc_labels_2021-06-19.parquet  cc_labels_2021-10-08.parquet
cc_labels_2021-06-20.parquet  cc_labels_2021-10-09.parquet
cc_labels_2021-06-21.parquet  cc_labels_2021-10-10.parquet
cc_labels_2021-06-29.parquet  cc_labels_2021-10-11.parquet
cc_labels_2021-06-30.parquet  cc_labels_2021-10-19.parquet
cc_labels_2021-07-01.parquet  cc_labels_2021-10-20.parqu

In [12]:
df = pd.read_parquet("data/train/madrid/labels/cc_labels_2021-06-01.parquet")

In [13]:
df.head()

,u,v,day,t,cc
0,171946,26513145,2021-06-01,0,2
1,171946,26513145,2021-06-01,1,1
2,171946,26513145,2021-06-01,2,1
3,171946,26513145,2021-06-01,4,1
4,171946,26513145,2021-06-01,5,1


In [14]:
import pickle

In [15]:
with open("data/london.pkl", "rb") as f:
    smth = pickle.load(f)

In [17]:
smth.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [22]:
import numpy as np

In [27]:
len(list(smth.values())[0])

629

In [28]:
len(list(smth.values()))

20

In [3]:
df_edges = pd.read_parquet("../data/road_graph/london/road_graph_edges.parquet")

In [4]:
df_edges.shape

(132414, 11)

In [5]:
df_edges.head()

,u,v,parsed_maxspeed,speed_kph,importance,highway,oneway,lanes,tunnel,length_meters,counter_distance
0,78112,25508583,32.2,32.2,0,unclassified,False,,,19.402386,6
1,78112,25508584,32.2,32.2,0,unclassified,False,,,63.881347,4
2,78112,3257621681005534125,32.2,32.2,0,residential,True,,,82.385612,5
3,99936,2146383887,32.2,32.2,0,unclassified,False,,,8.076410,1
4,99936,4544836433,32.2,32.2,0,unclassified,True,,,46.803240,0


In [20]:
df_edges.to_dict("records")[0]

{'u': 78112,
 'v': 25508583,
 'parsed_maxspeed': 32.2,
 'speed_kph': 32.2,
 'importance': 0,
 'highway': 'unclassified',
 'oneway': False,
 'lanes': '',
 'tunnel': '',
 'length_meters': 19.402385843568535,
 'counter_distance': 6}

In [6]:
df_nodes = pd.read_parquet("../data/road_graph/london/road_graph_nodes.parquet")

In [8]:
df_nodes.shape

(59110, 5)

In [9]:
df_nodes.head()

,node_id,counter_info,num_assigned,x,y
0,78112,,,-0.145792,51.526976
1,99936,,,-0.152791,51.523611
2,99937,,,-0.152024,51.523018
3,101818,01/285,1,-0.148104,51.535179
4,101831,02/065,1,-0.147044,51.535612


# Breaking down road graph mapping

In [24]:
edge_records = df_edges.to_dict("records")
edges = [(r["u"], r["v"]) for r in edge_records]

In [25]:
noncounter_nodes = [r["node_id"] for r in df_nodes.to_dict("records") if r["counter_info"] == ""]
counter_nodes = [r["node_id"] for r in df_nodes.to_dict("records") if r["counter_info"] != ""]
nodes = counter_nodes + noncounter_nodes

In [27]:
from collections import defaultdict

In [28]:
node_to_int_mapping = defaultdict(lambda: -1)
for i, k in enumerate(nodes):
    node_to_int_mapping[k] = i

In [32]:
edge_index = torch.tensor(
    [[node_to_int_mapping[n] for n, _ in edges],
     [node_to_int_mapping[n] for _, n in edges]], dtype=torch.long
)

In [33]:
edge_index_d = defaultdict(lambda: -1)
for i, (u, v) in enumerate(edges):
    edge_index_d[(u, v)] = i

In [25]:
edge_attributes=["speed_kph", "parsed_maxspeed", "length_meters", "counter_distance",
                                                 "importance", "highway", "oneway", ]

In [36]:
edge_attr = None

In [51]:
if edge_attributes is not None:
    edge_attr = torch.full(size=(len(edges), len(edge_attributes)), fill_value=float("nan"))
    highway_dict = {}
    oneway_dict = {}
    
    for i, edge in enumerate(edge_records):
        for j, attr in enumerate(edge_attributes):
            if attr == "highway":
                if edge[attr] not in highway_dict:
                    temp = len(highway_dict)
                    highway_dict[edge[attr]] = temp
                edge_attr[i, j] = highway_dict[edge[attr]]
            elif attr == "oneway":
                if edge[attr] not in oneway_dict:
                    temp = len(oneway_dict)
                    oneway_dict[edge[attr]] = temp
                edge_attr[i, j] = oneway_dict[edge[attr]]
            else:
                edge_attr[i, j] = edge[attr]

In [52]:
edge_attr[:5]

tensor([[32.2000, 32.2000, 19.4024,  6.0000,  0.0000,  0.0000,  0.0000],
        [32.2000, 32.2000, 63.8813,  4.0000,  0.0000,  0.0000,  0.0000],
        [32.2000, 32.2000, 82.3856,  5.0000,  0.0000,  1.0000,  1.0000],
        [32.2000, 32.2000,  8.0764,  1.0000,  0.0000,  0.0000,  0.0000],
        [32.2000, 32.2000, 46.8032,  0.0000,  0.0000,  0.0000,  1.0000]])

In [66]:
edge_attr[:, 4].unique()

tensor([0., 1., 2., 3., 4., 5.])

In [45]:
highway_dict

{'unclassified': 0,
 'residential': 1,
 'trunk': 2,
 'primary': 3,
 'secondary': 4,
 'secondary_link': 5,
 'tertiary': 6,
 'primary_link': 7,
 "['tertiary', 'residential']": 8,
 'motorway': 9,
 'motorway_link': 10,
 "['secondary', 'residential']": 11,
 "['residential', 'unclassified']": 12,
 'tertiary_link': 13,
 'trunk_link': 14,
 "['tertiary', 'trunk_link']": 15,
 "['secondary', 'unclassified']": 16,
 "['motorway', 'motorway_link']": 17,
 "['primary', 'primary_link']": 18,
 "['tertiary', 'unclassified']": 19,
 "['secondary', 'tertiary']": 20,
 "['primary', 'unclassified']": 21,
 "['residential', 'trunk_link']": 22,
 "['motorway_link', 'unclassified']": 23,
 "['primary', 'trunk_link']": 24,
 "['primary', 'residential']": 25,
 "['residential', 'primary']": 26}

In [58]:
from t4c22.t4c22_config import cc_dates, load_basedir

In [59]:
basedir = load_basedir()

In [61]:
cc_dates(basedir, "london")[:5]

['2019-07-03', '2019-07-20', '2019-11-18', '2020-01-16', '2019-10-22']

In [63]:
import pickle
cluster_map = {}
with open("data/london.pkl", 'rb') as f:
    maps = pickle.load(f)
for i in range(20):
    for day, t, _ in maps[i]:
        cluster_map['%s-%d' % (day, t)] = i

In [65]:
len(cluster_map)

7140

In [28]:
from t4c22.dataloading.road_graph_mapping import TorchRoadGraphMapping
from t4c22.t4c22_config import day_t_filter_weekdays_daytime_only

In [29]:
torch_road_graph_mapping = TorchRoadGraphMapping(
    city="london",
    edge_attributes=edge_attributes,
    root=basedir,
    df_filter=day_t_filter_weekdays_daytime_only
)

# load_inputs_day_t function from raod_graph_mapping

In [16]:
from t4c22.t4c22_config import load_inputs, load_basedir

In [18]:
basedir = load_basedir()

In [80]:
day = "2020-01-27"
t = 30
df_x = load_inputs(basedir, city="london", split="train", day=day)

In [81]:
df_x.head()

,node_id,day,t,volumes_1h
2420,101818,2020-01-27,24,"[nan, nan, nan, nan]"
2421,101818,2020-01-27,25,"[nan, nan, nan, nan]"
2422,101818,2020-01-27,26,"[nan, nan, nan, nan]"
2423,101818,2020-01-27,27,"[nan, nan, nan, nan]"
2424,101818,2020-01-27,28,"[nan, nan, nan, nan]"


In [79]:
df_x.t.unique()

array([24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
       75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87])

In [82]:
df_x["node_id"] = df_x["node_id"].astype("int64")
df_x = df_x.explode("volumes_1h")
df_x.head()

,node_id,day,t,volumes_1h
2420,101818,2020-01-27,24,NaN
2420,101818,2020-01-27,24,NaN
2420,101818,2020-01-27,24,NaN
2420,101818,2020-01-27,24,NaN
2421,101818,2020-01-27,25,NaN


In [83]:
df_x = df_x.reset_index()
df_x["slot"] = df_x.index % 4
df_x["volumes_1h"] = df_x["volumes_1h"].astype("float")
df_x.head()

,index,node_id,day,t,volumes_1h,slot
0,2420,101818,2020-01-27,24,NaN,0
1,2420,101818,2020-01-27,24,NaN,1
2,2420,101818,2020-01-27,24,NaN,2
3,2420,101818,2020-01-27,24,NaN,3
4,2421,101818,2020-01-27,25,NaN,0


In [84]:
x = torch.full(size=(len(torch_road_graph_mapping.node_to_int_mapping), 4), fill_value=float("nan"))
x.shape

torch.Size([59110, 4])

In [86]:
# (Mis-)use (day,t) for dataloading test sets where we do not exhibit day,t
if day == "test":
    data = df_x[(df_x["test_idx"] == idx)].copy()
else:
    data = df_x[(df_x["day"] == day) & (df_x["t"] == t)].copy()
data.head()

,index,node_id,day,t,volumes_1h,slot
24,2426,101818,2020-01-27,30,NaN,0
25,2426,101818,2020-01-27,30,NaN,1
26,2426,101818,2020-01-27,30,306.0,2
27,2426,101818,2020-01-27,30,224.0,3
280,5398,101831,2020-01-27,30,304.0,0


In [87]:
data["node_index"] = [torch_road_graph_mapping.node_to_int_mapping[x] for x in data["node_id"]]

In [88]:
x[data["node_index"].values, data["slot"].values] = torch.tensor(data["volumes_1h"].values).float()

In [90]:
x

tensor([[ nan,  nan, 306., 224.],
        [304., 347., 408., 452.],
        [230., 274., 293., 298.],
        ...,
        [ nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan]])

In [91]:
x.shape

torch.Size([59110, 4])